# Data Merging

In [27]:
import datetime, time, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import json

from cesium import datasets

from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from datetime import datetime, timezone, timedelta

resample_size = "15MIN"
resample_factor = 15

#windowfactor = 15

import warnings

In [28]:
df_merged_1 = pd.read_pickle('data/pickle/df_merged_1.pickle')
df_merged_2 = pd.read_pickle('data/pickle/df_merged_2.pickle')
df_merged_3 = pd.read_pickle('data/pickle/df_merged_3.pickle')


dfs = [df_merged_1, df_merged_2, df_merged_3]

In [29]:
df_merged_3

,sechs_h_regelung,epex_da_de_eur_mwh,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,...,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y,50Hertz_power_mw,DE_power_mw,DK_power_mw,DK1_power_mw,TTG_power_mw
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2016-12-31 23:00:00,0.000000e+00,20.960000,0.0,0.0,0.0,0.0,0.0,6291.0,40241.0,2942.0,...,331.0,3340.0,1004.0,652.0,3009.0,4922.0,12966.0,2282.0,1857.0,6636.0
2016-12-31 23:15:00,2.943790e-88,20.463647,0.0,0.0,0.0,0.0,0.0,6134.0,39805.0,2942.0,...,331.0,3338.0,1004.0,652.0,3007.0,4890.0,12916.0,2282.0,1857.0,6614.0
2016-12-31 23:30:00,2.667555e-88,20.622226,0.0,0.0,0.0,0.0,0.0,5976.0,39023.0,2942.0,...,331.0,3339.0,1004.0,652.0,3008.0,4859.0,12873.0,2282.0,1857.0,6595.0
2016-12-31 23:45:00,1.057543e-88,20.934692,0.0,0.0,0.0,0.0,0.0,5871.0,38682.0,2942.0,...,331.0,3339.0,1004.0,652.0,3008.0,4833.0,12830.0,2282.0,1857.0,6572.0
2017-01-01 00:00:00,0.000000e+00,20.900000,0.0,0.0,0.0,0.0,0.0,5810.0,38162.0,2735.0,...,331.0,3370.0,1094.0,748.0,3039.0,4808.0,12315.0,2152.0,1774.0,6078.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-13 20:00:00,0.000000e+00,95.030000,0.0,0.0,0.0,0.0,0.0,10294.0,53885.0,3505.0,...,300.0,744.0,226.0,127.0,444.0,3547.0,8037.0,274.0,193.0,2429.0
2021-07-13 20:15:00,0.000000e+00,93.350088,0.0,0.0,0.0,0.0,0.0,10083.0,53276.0,3505.0,...,308.0,746.0,226.0,127.0,438.0,3578.0,8140.0,274.0,193.0,2497.0
2021-07-13 20:30:00,0.000000e+00,91.301784,0.0,0.0,0.0,0.0,0.0,9899.0,52355.0,3505.0,...,315.0,739.0,226.0,127.0,424.0,3608.0,8239.0,274.0,193.0,2562.0


In [30]:
df_merged_4 = reduce(lambda  left,right: pd.merge(left, right, on=['dt_start_utc'],
                                            how='inner'), dfs)

In [31]:
count_inf = np.isinf(df_merged_4).values.sum()
count_nan = df_merged_4.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 182 missing values.


In [32]:
df_merged_4.shape

(17368, 63)

In [33]:
df_merged_4.dropna(inplace=True)

In [34]:
df_merged_4.shape

(17358, 63)

In [35]:
count_inf = np.isinf(df_merged_4).values.sum()
count_nan = df_merged_4.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 0 missing values.


In [36]:
df_merged_4.to_pickle('data/pickle/df_merged_4.pickle')

In [37]:
df_merged_4.to_csv("df_merged_4.csv")

In [38]:
df_merged_4.shape

(17358, 63)

In [39]:
df_merged_4.head()

,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,total_pred_cons,Biomasse[MWh],Wasserkraft[MWh],Wind Offshore[MWh],Wind Onshore[MWh],...,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2021-01-01 02:30:00,1051.0,2590.0,131.0,54.0,854.0,4680.0,1.138,295.0,95.0,890.0,...,175.0,395.0,172.0,71.0,220.0,1157.0,2835.0,153.0,61.0,982.0
2021-01-01 02:45:00,1027.0,2536.0,131.0,54.0,827.0,4575.0,1.139,295.0,90.0,860.0,...,174.0,395.0,172.0,71.0,221.0,1127.0,2764.0,153.0,61.0,943.0
2021-01-01 03:00:00,1004.0,2486.0,105.0,43.0,801.0,4439.0,1.138,294.0,80.0,843.0,...,172.0,393.0,145.0,63.0,221.0,1098.0,2699.0,131.0,54.0,909.0
2021-01-01 03:15:00,989.0,2452.0,105.0,43.0,783.0,4372.0,1.137,290.0,76.0,833.0,...,166.0,382.0,145.0,63.0,216.0,1073.0,2643.0,131.0,54.0,882.0
2021-01-01 03:30:00,978.0,2423.0,105.0,43.0,765.0,4314.0,1.139,292.0,73.0,812.0,...,161.0,371.0,145.0,63.0,210.0,1051.0,2590.0,131.0,54.0,854.0
